In [2]:
import pandas as pd
import os
import dask.dataframe as dd

pd.set_option("mode.copy_on_write", True)

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
import os

print(f"I have {os.cpu_count()} logical cores.")

I have 8 logical cores.


In [4]:
# Define the columns you want to keep
required_columns = [
    "BUYER_COUNTY",
    "BUYER_STATE",
    "TRANSACTION_DATE",
    "DOSAGE_UNIT",
    "CALC_BASE_WT_IN_GM",
    "MME",
]

large_df = dd.read_csv(
    "/Users/keonnartey/Downloads/arcos_all_washpost.zip",
    compression="zip",
    sep="\t",
    usecols=required_columns,
    dtype={
        "MME": "float64",
        "CALC_BASE_WT_IN_GM": "float64",
    },  # Specify the dtype for the columns
)

# Extract year
large_df["Year"] = dd.to_datetime(large_df.TRANSACTION_DATE, format="%Y-%m-%d").dt.year

# Group by BUYER_COUNTY, BUYER_STATE, Year and sum MME and CALC_BASE_WT_IN_GM

grouped_df = (
    large_df.groupby(["BUYER_COUNTY", "BUYER_STATE", "Year"])["MME"].sum().reset_index()
)

/Users/keonnartey/miniconda3/envs/dask_env/lib/python3.11/site-packages/dask/dataframe/io/csv.py:546: UserWarning: Warning zip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


In [5]:
result = grouped_df.compute()
result.head()